Install nuget package
- install semantic kernel 1.22.0
- install MistralAI connector 1.22.0-alpha
- install OpenApi Plugin 1.22.0-alpha

In [7]:
#r "nuget: Microsoft.SemanticKernel, 1.22.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.MistralAI, 1.22.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.OpenApi, 1.22.0-alpha"

Installed Packages Microsoft.SemanticKernel, 1.22.0 Microsoft.SemanticKernel.Connectors.MistralAI, 1.22.0-alpha Microsoft.SemanticKernel.Plugins.OpenApi, 1.22.0-alpha

import packages

In [8]:

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.MistralAI;
using Microsoft.SemanticKernel.Plugins.OpenApi;

Create Semantic Kernel builder

In [9]:
// create and build Kernel using Mistral AI configuration
#pragma warning disable SKEXP0070
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddMistralChatCompletion(
    modelId: "mistral-small-latest",
    apiKey: "SpUM9gQIh7TDJTMBIVay3IXkH5yrfYiU"
);
Kernel kernel = kernelBuilder.Build();
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

Add plugin to Kernel

In [10]:
// Add a plugin (openai plugin)

#pragma warning disable SKEXP0040
await kernel.ImportPluginFromOpenApiAsync(
   pluginName: "lights",
   uri: new Uri("https://localhost:7082/swagger/v1/swagger.json"),
   executionParameters: new OpenApiFunctionExecutionParameters()
   {
      // Determines whether payload parameter names are augmented with namespaces.
      // Namespaces prevent naming conflicts by adding the parent parameter name
      // as a prefix, separated by dots
      EnablePayloadNamespacing = true
   }
);


Call ChatCompletion service and get response
- Enable Planner
    - Define Execution Setting for Mistral AI (ToolCallBehavior Auto)
- Create chat prompt message (input) and invoke kernel

In [11]:

#pragma warning disable SKEXP0070
// Enable planning with Mistral AI settings
var execution_settings = new MistralAIPromptExecutionSettings
{
    ToolCallBehavior = MistralAIToolCallBehavior.AutoInvokeKernelFunctions
};

//User input
const string chatPrompt = "<message role=\"user\"> Gets the Weather Forecast</message>";

var chatSemanticFunction = kernel.CreateFunctionFromPrompt(chatPrompt,execution_settings);

// Get the response from the AI
var chatPromptResult = await kernel.InvokeAsync(chatSemanticFunction);
// Print the results
Console.WriteLine("Assistant > " + chatPromptResult);

Assistant > Here is the weather forecast for the next 5 days:

- October 11, 2024: Hot 29°C (84°F)
- October 12, 2024: Warm 47°C (116°F)
- October 13, 2024: Chilly 34°C (93°F)
- October 14, 2024: Cool 43°C (109°F)
- October 15, 2024: Scorching 6°C (42°F)
